In [1]:
import sys
import os
import tensorflow
from tensorflow.keras.models import load_model
print(os.getcwd())

/home/dennis/Documents/McMaster_eHealth_Masters_2018-19/SEP789-C02_DeepLearning/wk3


In [2]:
# constants
logdir = os.path.join(os.getcwd(), 'training_logs')

In [ ]:
help(tensorflow.keras.callbacks.TensorBoard)

In [3]:
# Example of LSTM to learn a sequence
# credit: https://machinelearningmastery.com/5-step-life-cycle-long-short-term-memory-models-keras/ 
from pandas import DataFrame
from pandas import concat
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import TensorBoard

# create sequence
length = 10
sequence = [i/float(length) for i in range(length)]
print(sequence)

# create X/y pairs
df = DataFrame(sequence)
df = concat([df.shift(1), df], axis=1)
df.dropna(inplace=True)

# convert to LSTM friendly format
values = df.values
X, y = values[:, 0], values[:, 1]
X = X.reshape(len(X), 1, 1)

# 1. define network
model = Sequential()
model.add(LSTM(10, input_shape=(1,1)))
model.add(Dense(1))

tb_callback = TensorBoard(logdir)

# 2. compile network
model.compile(optimizer='adam', loss='mean_squared_error', callbacks=[tb_callback])

# 3. fit network
history = model.fit(X, y, epochs=1000, batch_size=len(X), verbose=0)

# 4. evaluate network
loss = model.evaluate(X, y, verbose=0)
print(loss)

# 5. make predictions
predictions = model.predict(X, verbose=0)
print(predictions[:, 0])

#6. save the model weights and parameters
model.save("tutorial_model.h5")

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


ValueError: Session keyword arguments are not support during eager execution. You passed: {'callbacks': [<tensorflow.python.keras.callbacks.TensorBoard object at 0x7f4f7998f080>]}

In [ ]:
# print out the model summary
model.summary()

In [ ]:
# print out the model summary as a diagram
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='tutorial_model.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Delete the model itself but load back the model weights
del model

In [ ]:
# load back the model weights that we have saved and 
model = load_model(os.path.join(os.getcwd(), 'model_architecture_outputs', 'tutorial_model.h5'))

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
import sys
print(sys.argv[0])
print(sys.argv)
import json

In [ ]:
model_json = model.to_json()
with open('tutorial_model.json', 'w') as fp:
     json.dump(model_json, fp)

In [ ]:
model.save_weights('tutorial_model_weights.h5')

In [ ]:
model_1 = model.load_weights(os.path.join(os.getcwd(), 'model_architecture_outputs', 'tutorial_model_weights.h5'))

In [ ]:
# If you want to apply transfer learning through having pre-trained model weights 
# loaded the model from the previously saved model into our new model
# previous model's name was 'model'

model_new_architecture_same_weights = Sequential()
model_new_architecture_same_weights.add(Dense(2, input_dim=3, name='dense_1'))  # will be loaded
model_new_architecture_same_weights.add(Dense(10, name='new_dense'))  # will not be loaded

model_new_architecture_same_weights = model.load_weights(os.path.join(os.getcwd(), 
                                                                      'model_architecture_outputs',
                                                                      'tutorial_model_weights.h5'),
                                                         by_name=True)

In [ ]:
# How can I obtain the output of an intermediate layer?
# One simple way is to create a new Model that will output the layers that you are interested in:
# Only works if you have given your model layer a name

from tensorflow.keras.models import Model
# model = ... create the original model

data = X.copy()

layer_name = 'dense_1'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(data)